## Getting Started

In [ ]:
from arcgis.gis import GIS
from arcgis.geoenrichment import enrich
from arcgis.geoenrichment import create_report
from arcgis.geometry import Envelope
from arcgis.features.manage_data import generate_tessellation
from arcgis.features import FeatureSet
from getpass import getpass

In [ ]:
password=getpass()
gis = GIS(username="YOUR-USERNAME", password=password)

## Build Analysis Areas

In [ ]:
inland_empire_map = gis.map('Redlands, CA, USA', zoomlevel=12)
inland_empire_map.basemap = 'openstreetmap'
inland_empire_map

In [ ]:
hexagons = generate_tessellation(None, bin_size=1, bin_size_unit='SquareMiles', bin_type='HEXAGON', context={"extent":inland_empire_map.extent})
inland_empire_map.add_layer(hexagons)

len(hexagons.layer.featureSet.features)

In [ ]:
geometry_areas = [{"geometry":f["geometry"].copy()} for f in hexagons.layer.featureSet.features]

for area in geometry_areas:
    area["geometry"]["spatialReference"]={"wkid":3857}

In [ ]:
enriched_areas=enrich(study_areas=geometry_areas, analysis_variables=["KeyGlobalFacts.TOTPOP","AtRisk.SENIORS_CY"])
enriched_areas.head()

In [ ]:
enriched_areas["SENIORS_PERCENT"] = enriched_areas["SENIORS_CY"]  / enriched_areas["TOTPOP"] * 100

senior_populated_area = enriched_areas[enriched_areas["SENIORS_PERCENT"] > 25]
senior_populated_area.shape[0]

In [ ]:
inland_empire_map2 = gis.map('Redlands, CA, USA', zoomlevel=12)
inland_empire_map2.basemap = 'openstreetmap'

inland_empire_map2.add_layer(FeatureSet.from_dataframe(senior_populated_area.copy()))
inland_empire_map2

In [ ]:
inland_empire_map2.add_layer(FeatureSet.from_dataframe(enriched_areas), {'renderer':'ClassedSizeRenderer','field_name':'SENIORS_PERCENT', 'opacity':0.5})

In [ ]:
most_senior_area = enriched_areas[enriched_areas.SENIORS_PERCENT == enriched_areas.SENIORS_PERCENT.max()]
most_senior_area

In [ ]:
fill_symbol = {
  "type": "esriSFS",
  "style": "esriSFSSolid",
  "color": [255,0,0,255],
    "outline": {
     "type": "esriSLS",
     "style": "esriSLSSolid",
     "color": [0,0,0,255],
     "width": 1
    }
}

inland_empire_map2.draw(most_senior_area.iloc[0].SHAPE, symbol = fill_symbol)

## Building Report

In [ ]:
most_senior_study_area = {"geometry":most_senior_area.iloc[0].SHAPE}

# coronavirus impact planning report
report = create_report(study_areas=[most_senior_study_area], report={"itemid":"ac36af7d86b74071a1cff5dc11f52ae8"}, out_name="COVID Impact Planning Profile.pdf", out_folder="output")

In [ ]:
# health care & insurace 
report = create_report(study_areas=[most_senior_study_area], report={"itemid":"d29fcd15f66f46edad86499675b0ec03"}, export_format='html', out_name="Health Care and Insurance.html", out_folder="output")